# Introduction
We'll be applying naive bayes to determine whether a tweet was written by [Donald Trump](https://twitter.com/realDonaldTrump) or [Hillary Clinton](https://twitter.com/HillaryClinton). For a more in-depth theoretical explanation of the theory behind the classifier, see [our notes](https://github.com/MachinesWhoLearn/lectures/blob/master/2016-2017.Meetings/05.DIY_naive_bayes/naive_bayes_primer/naive_bayes_primer.pdf).

### Features
In our model, the features are the individual words. For example, we'd expect that the word "wall" would be more likely to appear in Trump tweets, so we want to account for that in our model. To start off with, we'll load the text data and then perform some basic tokenization to separate out all the words.

# Tokenizing and Counting the Words
We want to transform our collection of tweets into something that the model can understand. A basic idea from natural language processing (NLP) is the bag of words approach (BOW). When we use the bag of words, we simply count the number of times a word occurs in a document, and divide it by the total number of words. Doing this for each word, we can get a probability distribution for the probability of a word occurring.

Let's start by loading our training data (the raw tweets) and the associated labels (indicating authorship, "0" for Hillary and "1" for Trump). We'll then count up all the statistics we'll need to use later for calculating probabilities

In [1]:
# setting up some imports and some miscellaneous helper methods we'll be using
import re
import string
from math import log
from __future__ import print_function

# our default smoothing value, we pretend we see each word at
# least once so all our probabilities are well-formed 
# (so no multiplying by 0 if a word isn't in train set)
SMOOTHING = 1.0

# location of data file relative to this notebook path
TRAIN_DATA_PATH = "../data/tweets_train.txt"
TRAIN_LABELS_PATH = "../data/labels_train.txt"
TEST_DATA_PATH = "../data/tweets_test.txt"
TEST_LABELS_PATH = "../data/labels_test.txt"

# strip punctuation from a string
def remove_punctuation(input_str):
    "from http://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python"
    table = string.maketrans("","")
    return input_str.translate(table, string.punctuation)

# split a line into tokens, or a list of words (known as tokens) 
# that are separated by whitespace
def tokenize(line):
    line = remove_punctuation(line)
    line = line.lower().strip()
    # re.split has an odd tendency to add empty strings, remove those
    return [token for token in re.split("\W+", line) if token != '']

In [2]:
# Setting up some the data structures we'll use to keep track of counts

# this dictionary maps the numeric label to the candidate name
# the "\n" after each label accounts for the newline character 
# that follows each one
LABEL_MAP = {'0\n': 'hillary', '1\n': 'trump'}

# this dictionary keeps track of the number of times
# word appears in a trump or hillary tweet
word_counts_per_candidate = {
    "trump": {},
    "hillary": {}
}
# this dictionary keeps track of how many trump tweets there are and how many
# hillary tweets there are. These are our priors!
total_tweet_count = {'trump': 0.0, 'hillary': 0.0}

# counts words across all tweets
vocabulary = {}

Let's take a moment to verify that `tokenize` does what we want it to.

In [3]:
sample_tweet = "I am growing the Republican Party tremendously - just look at the numbers, way up! Democrats numbers are significantly down from years past."
sample_tokens = tokenize(sample_tweet)
sample_tokens

['i',
 'am',
 'growing',
 'the',
 'republican',
 'party',
 'tremendously',
 'just',
 'look',
 'at',
 'the',
 'numbers',
 'way',
 'up',
 'democrats',
 'numbers',
 'are',
 'significantly',
 'down',
 'from',
 'years',
 'past']

In [4]:
# open the file with train data and refer to it as "data", and
# open the file with train labels and refer to it as "labels"
with open(TRAIN_DATA_PATH, 'r') as data, open(TRAIN_LABELS_PATH, 'r') as labels:
    # zip data and labels together to create a list of tuples of (tweet,label),
    # which we then iterate over. the tweet variable refers to an individual tweet
    # and the label variable refers to an individual label
    for tweet, label in zip(data, labels):
        # if it is a Hillary tweet, increment the "hillary" element of `total_tweet_count`
        # if it is a Trump tweet, increment the "trump" element of `total_tweet_count`.
        total_tweet_count[LABEL_MAP[label]] += 1.0
        
        tokens = tokenize(tweet)
        
        # iterate over the words we found in the tweet
        for word in tokens:
            # if we haven't seen a word yet, let's add it to our dictionary with a 
            # count of 2 * how much we smoothing up by (`SMOOTHING`),
            # (since we presume that both hillary and trump have seen it)
            # we'll be incrementing it later
            if word not in vocabulary:
                vocabulary[word] = 4 * SMOOTHING
                # similarily, since we pretend that we saw an extra tweet 
                # with this unknown word and one without, we add 2*SMOOTHING
                # to hillary and trump
                total_tweet_count["trump"] += 2*SMOOTHING
                total_tweet_count["hillary"] += 2*SMOOTHING
                word_counts_per_candidate["trump"][word] = 1
                word_counts_per_candidate["hillary"][word] = 1
            # now add one to the global dictionary
            # and the associated candidate's
            # dictionary
            vocabulary[word] += 1
            word_counts_per_candidate[LABEL_MAP[label]][word] += 1


Let's take a look at our output / a bit of our output for the long dictionaries

In [5]:
total_tweet_count

{'hillary': 16812.0, 'trump': 16770.0}

In [6]:
first_10_vocabulary = {k: vocabulary[k] for k in vocabulary.keys()[:10]}
first_10_vocabulary

{'electricity': 5.0,
 'eligible': 5.0,
 'foul': 5.0,
 'four': 29.0,
 'igual': 5.0,
 'looking': 39.0,
 'lord': 5.0,
 'plaudits': 5.0,
 'railing': 6.0,
 'rawlingsblake': 5.0}

In [7]:
first_10_word_counts_per_candidate_H = {k: word_counts_per_candidate["hillary"][k] for k in word_counts_per_candidate["hillary"].keys()[:10]}
first_10_word_counts_per_candidate_H

{'electricity': 1,
 'eligible': 2,
 'foul': 1,
 'four': 11,
 'igual': 2,
 'looking': 7,
 'lord': 1,
 'plaudits': 1,
 'railing': 2,
 'rawlingsblake': 1}

In [8]:
first_10_word_counts_per_candidate_T = {k: word_counts_per_candidate["trump"][k] for k in word_counts_per_candidate["trump"].keys()[:10]}
first_10_word_counts_per_candidate_T

{'electricity': 2,
 'eligible': 1,
 'foul': 2,
 'four': 16,
 'igual': 1,
 'looking': 30,
 'lord': 2,
 'plaudits': 2,
 'railing': 2,
 'rawlingsblake': 2}

# Calculating Conditional Probabilities
Now, we'll proceed to calculate the the conditional probabilities for each word; that is, we want to know $\mathbb{P}(word|candidate)$, for each candidate. More verbosely, we will be calculating $\mathbb{P}(word|H)$ and $\mathbb{P}(word|T)$ for each in our vocabulary and for each event $H$ and $T$ (where $H$ is the event that Hillary Clinton is the author, and $T$ is the event that Donald Trump is the author).

In [9]:
word_probabilities_per_candidate = {
    "trump": {},
    "hillary": {}
}

# iterate over the hillary and trump dictionaries within 
# `word_count_per_candidate` with this outside loop
for candidate in word_counts_per_candidate:
    for word, count in word_counts_per_candidate[candidate].items():
        word_probabilities_per_candidate[candidate][word] = count / total_tweet_count[candidate]

Now let's take a look at our calculated probabilities

In [10]:
first_10_word_probabilities_per_candidate_H = {k: word_probabilities_per_candidate["hillary"][k] for k in word_probabilities_per_candidate["hillary"].keys()[:10]}
first_10_word_probabilities_per_candidate_H

{'electricity': 5.948132286462051e-05,
 'eligible': 0.00011896264572924102,
 'foul': 5.948132286462051e-05,
 'four': 0.0006542945515108256,
 'goodpaying': 0.00035688793718772306,
 'igual': 0.00011896264572924102,
 'looking': 0.0004163692600523436,
 'lord': 5.948132286462051e-05,
 'plaudits': 5.948132286462051e-05,
 'railing': 0.00011896264572924102}

In [11]:
first_10_word_probabilities_per_candidate_T = {k: word_probabilities_per_candidate["trump"][k] for k in word_probabilities_per_candidate["trump"].keys()[:10]}
first_10_word_probabilities_per_candidate_T

{'electricity': 0.00011926058437686345,
 'eligible': 5.9630292188431724e-05,
 'foul': 0.00011926058437686345,
 'four': 0.0009540846750149076,
 'goodpaying': 5.9630292188431724e-05,
 'igual': 5.9630292188431724e-05,
 'looking': 0.0017889087656529517,
 'lord': 0.00011926058437686345,
 'plaudits': 0.00011926058437686345,
 'railing': 0.00011926058437686345}

# Applying the Classifier
In this problem, the posterior probability of a tweet, represented as a set of words $(x_1, \dots, x_n)$, being written by Hillary is defined with Bayes' rule as: $$\mathbb{P}(H|x_1, \dots, x_n) \approx \frac{\mathbb{P}(H)\prod\limits_{i=1}^n \mathbb{P}(x_i | H)}{\mathbb{P}(H)\prod\limits_{i=1}^n \mathbb{P}(x_i | H) + \mathbb{P}(T)\prod\limits_{i=1}^n \mathbb{P}(x_i | T)}$$

We now have all the tools we need to calculate this posterior probability of a tweet being written by Hillary Clinton or Donald Trump. We'll walk through what everything represents step by step.

$\mathbb{P}(H)$ is the prior probability of a tweet being written by Hillary, which is calculated with $\frac{\# of Hillary tweets}{\# of total tweets}$. This information is in the `total_tweet_count` dictionary. Similarly, we can calculate $\mathbb{P}(T)$, the prior probability of a tweet being written by Trump, which is $\frac{\# of Trump tweets}{\# of total tweets}$.

$\prod\limits_{i=1}^n \mathbb{P}(x_i | H)$ is the product of the conditional probabilities for each word $\mathbb{P}(x_i | H)$. We've calculated $\mathbb{P}(x_i | H)$ for each word in the dictionary `word_probabilities_per_candidate`, so we can simply multiply them together to get their product. We take a similar approach with $\prod\limits_{i=1}^n \mathbb{P}(x_i | T)$.


## Log-Probabilities
Since we're multiplying many small values together, we run the risk of floating-point underflow. We solve this, in short, by turning $a \times b$ into $log(a) + log(b)$. Since converting the probabilities to log values does nothing to their ordering (if $\mathbb{P}(A) > \mathbb{P}(B)$, then $log(\mathbb{P}(A)) > log(\mathbb{P}(B)$)). In this way, we avoid arithmetic underflow while keeping the well-ordering of the probabilities, which is all we need for classification.

In [12]:
# create a new counter to keep track of how many we classified correctly
# and incorrectly. tuples are arranged (our guess, correct label)
outcomes = {("trump", "trump"): 0.0,
            ("trump", "hillary"): 0.0,
            ("hillary", "hillary"): 0.0,
            ("hillary", "trump"): 0.0
           }

# calculate our priors and take the log
prior_trump = total_tweet_count['trump'] / sum(total_tweet_count.values())
prior_hillary = total_tweet_count['hillary'] / sum(total_tweet_count.values())
log_prior_trump = log(prior_trump)
log_prior_hillary = log(prior_hillary)

# open the test data and test labels and refer to them as "test_data" and "test_labels", respectively
with open(TEST_DATA_PATH, 'r') as test_data, open(TEST_LABELS_PATH, 'r') as test_labels:
    # zip `test_data` and `test_labels` together to create a list of tuples of (tweet,label),
    # which we then iterate over. the `tweet` variable refers to an individual tweet
    # and the `label` variable refers to an individual label
    for test_tweet, test_label in zip(test_data, test_labels):
        # turn the test tweet into tokens
        test_tokens = tokenize(test_tweet)
        log_p_tweet_given_trump = 0.0
        log_p_tweet_given_hillary = 0.0
        for token in test_tokens:
            if token in vocabulary:
                p_word_given_trump = word_probabilities_per_candidate['trump'][token]
                log_p_word_given_trump = log(p_word_given_trump)
                # remember adding two logs is like multiplying their raw values
                log_p_tweet_given_trump += log_p_word_given_trump

                p_word_given_hillary = word_probabilities_per_candidate['hillary'][token]
                log_p_word_given_hillary = log(p_word_given_hillary)
                log_p_tweet_given_hillary += log_p_word_given_hillary
            else:
                # note that the token isn't in word_probabilities (and thus 
                # isn't seen in our train set), then we just ignore it
                # this works fine for this model, but there are a variety of NLP techniques
                # one could apply to handle "unknown tokens". Feel 
                # free to ask if you want details!
                pass
            
        # note that we don't actually have to do this, but it's here for didactic purposes
        # we can directly compare `log_p_tweet_given_hillary` with `log_p_tweet_given_trump`
        # to figure out which posterior probability will be greater!
        p_hillary_given_tweet_denominator = ((log_prior_hillary+log_p_tweet_given_hillary) + 
                                             (log_prior_trump+log_p_tweet_given_trump))
        # division is subtraction in logspace
        p_hillary_given_tweet = (log_prior_hillary+log_p_tweet_given_hillary) - p_hillary_given_tweet_denominator
        
        p_trump_given_tweet_denominator = ((log_prior_hillary+log_p_tweet_given_hillary) + 
                                           (log_prior_trump+log_p_tweet_given_trump))
        # division is subtraction in logspace
        p_trump_given_tweet = (log_prior_trump+log_p_tweet_given_trump) - p_trump_given_tweet_denominator
        
        # echoing what was said above, we could simply compare log_prior_trump+log_p_tweet_given_trump with
        # log_prior_hillary+log_p_tweet_given_hillary, since notice that the values of the denominators
        # in both cases for the full posterior probability is the same! thus, if you want to 
        # merely find out which is larger, you can just compare the numerators.
        if p_trump_given_tweet >= p_hillary_given_tweet:
            # probability that trump wrote the tweet is higher, 
            # so we predict trump. Write our prediction
            # and the correct label to the dictionary
            outcomes[("trump", LABEL_MAP[test_label])] += 1
        else:
            # probability that trump wrote the tweet is higher, 
            # so we predict trump. Write our prediction
            # and the correct label to the dictionary
            outcomes[('hillary', LABEL_MAP[test_label])] += 1

In [13]:
outcomes

{('hillary', 'hillary'): 756.0,
 ('hillary', 'trump'): 150.0,
 ('trump', 'hillary'): 43.0,
 ('trump', 'trump'): 640.0}

# Analyzing `Outcomes`
We see that we predicted "hillary" correctly 756 times, and predicted "trump" correctly 640 times. However, we made a fair amount of errors as well; confusing "trump" for "hillary" 150 times and vice versa 43 times.

Let's calculate our accuracy

In [14]:
accuracy = (outcomes[("hillary", "hillary")] + outcomes[("trump", "trump")]) / sum(outcomes.values())

print("accuracy: {}".format(accuracy))

accuracy: 0.87853996224
